## Задание
На сайте QIWI-кошелька, на форме сотовой связи был проведен АБ-тест, группы были сформированы случайным образом. Задача - ответить на вопрос команды - "Какой вариант лучше?"

## Подключение библиотек и скриптов

In [1]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.image as img
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


%matplotlib inline
%config InlineBackend.figure_format = 'png'

In [2]:
plt.style.use('seaborn-bright')
plt.rcParams['figure.figsize'] = (6, 4)
matplotlib.rcParams.update({'font.size': 14})

In [3]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 50)

In [4]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#### Путь к директориям и файлам

In [5]:
DATASET_PATH = r'/Users/Kristina/Desktop/EDA/test/test_qiwi.csv'

## Загрузка данных

- **datetime_of_event** - дата и время совершения события
- **user_id** - уникальный идентификатор пользователя
- **event** - события, фиксируются на клиенте (OpenPage, PaymentSuccess)
- **success_flag** - флаг успешной транзакции (1 - успешная, 0 - неуспешная)
- **test_group** - группа теста. А - контрольная группа(старая форма оплаты), B - тестовая группа(новая форма оплаты)
- **provider** - Оператор сотовой связи на который была произведена оплата (фиксируется только для успешных транзакций на сервере)
- **amount** - Сумма оплаты (фиксируется только для успешных транзакций на сервере)

In [6]:
df = pd.read_csv(DATASET_PATH)
df.head()

,datetime_of_event,user_id,event,success_flag,test_group,provider,amount
0,2016-06-13 23:31:57.800,548e37dfa2690defe92201412ff9b02f,OpenPage,0,A,NaN,nan
1,2016-06-07 04:14:20.700,2c2205ffc0501665f64bf0703185b6d8,OpenPage,0,A,NaN,nan
2,2016-06-09 23:40:11.700,0b89952e0a145ffc36ea063e401e475c,OpenPage,0,B,NaN,nan
3,2016-06-16 12:35:21.800,6b6cc68c9a10b163c147b08be78f1cb2,OpenPage,0,B,NaN,nan
4,2016-06-14 09:42:25.000,8a154919eefdcf48814a69da0b841e87,OpenPage,0,A,NaN,nan


In [7]:
df.shape

(97230, 7)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97230 entries, 0 to 97229
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   datetime_of_event  97230 non-null  object 
 1   user_id            97230 non-null  object 
 2   event              97230 non-null  object 
 3   success_flag       97230 non-null  int64  
 4   test_group         97229 non-null  object 
 5   provider           29930 non-null  object 
 6   amount             29930 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 5.2+ MB


In [9]:
df.nunique()

datetime_of_event    96106
user_id              68648
event                    2
success_flag             2
test_group               2
provider                 4
amount                3630
dtype: int64

In [10]:
df.isna().sum()

datetime_of_event        0
user_id                  0
event                    0
success_flag             0
test_group               1
provider             67300
amount               67300
dtype: int64

## Обзор данных

#### Категориальные переменные

In [11]:
for obj in df.select_dtypes(include='object').columns:
    print(str(obj) + '\n\n' + str(df[obj].value_counts()) + '\n' + '-' * 100 + '\n')

datetime_of_event

2016-06-15 11:10:54.900    3
2016-06-15 13:55:01.100    3
2016-06-11 19:51:49.900    3
2016-06-16 13:19:19.700    3
2016-06-16 18:55:42.700    3
                          ..
2016-06-11 10:39:46.300    1
2016-06-11 20:09:21.300    1
2016-06-07 20:25:23.500    1
2016-06-11 03:21:15.200    1
2016-06-11 19:13:52.600    1
Name: datetime_of_event, Length: 96106, dtype: int64
----------------------------------------------------------------------------------------------------

user_id

4118a432b767d136b1adb4d2a82ce32c    35
fa77b42f47b4b5ac8ec0803c04cd8298    22
88aa45358e24f366cc6b64d81680b70c    21
cd1a50c1af1b4089855e22dac3fd23f7    21
817a4b6e475770881fcf98bb1adf39df    19
                                    ..
9a9da365003a8ab1c75ed1d61a0a663c     1
57b760864a4485db9e44ce67a360242c     1
1b9fc80ded10f51cbf14fa9f14f91237     1
44147206799ad50cc427a9a6916c063e     1
3fe3cc9fa5f3a652ce0b1cfefa17e1fd     1
Name: user_id, Length: 68648, dtype: int64
--------------------------

#### Количественные переменные

In [12]:
df.describe()

,success_flag,amount
count,97230.00,29930.00
mean,0.31,435.78
std,0.46,906.70
min,0.00,1.00
25%,0.00,30.00
50%,0.00,100.00
75%,1.00,320.00
max,1.00,14998.00


In [13]:
df

,datetime_of_event,user_id,event,success_flag,test_group,provider,amount
0,2016-06-13 23:31:57.800,548e37dfa2690defe92201412ff9b02f,OpenPage,0,A,NaN,nan
1,2016-06-07 04:14:20.700,2c2205ffc0501665f64bf0703185b6d8,OpenPage,0,A,NaN,nan
2,2016-06-09 23:40:11.700,0b89952e0a145ffc36ea063e401e475c,OpenPage,0,B,NaN,nan
3,2016-06-16 12:35:21.800,6b6cc68c9a10b163c147b08be78f1cb2,OpenPage,0,B,NaN,nan
4,2016-06-14 09:42:25.000,8a154919eefdcf48814a69da0b841e87,OpenPage,0,A,NaN,nan
...,...,...,...,...,...,...,...
97225,2016-06-12 17:12:47.700,eed3ed6c01e59c37c6464e41bc066076,OpenPage,0,B,NaN,nan
97226,2016-06-15 05:40:57.200,33c9bb81b998ca0f1e8d24dd18273a54,OpenPage,0,B,NaN,nan
97227,2016-06-08 11:15:57.400,857fa51168a167166d244e760b6c0141,OpenPage,0,B,NaN,nan
97228,2016-06-11 18:24:07.600,3209ddc552fb40b183fade449f19ef85,OpenPage,0,A,NaN,nan


## Решение

### Обработка пропусков & выбросов

Выше видим, что есть одно незаполненное значение в %test_group% - убираем его из датасета

In [14]:
df = df[pd.notnull(df['test_group'])]

In [15]:
df['test_group'].isna().sum()

0

In [16]:
df['user_id'].unique().shape[0]

68647

In [17]:
df.groupby(['test_group', 'user_id']).size().reset_index(name='count') \
      .sort_values(['count'], ascending=[False])                                         \
      .reset_index(drop = True)

,test_group,user_id,count
0,A,4118a432b767d136b1adb4d2a82ce32c,35
1,A,fa77b42f47b4b5ac8ec0803c04cd8298,22
2,A,cd1a50c1af1b4089855e22dac3fd23f7,21
3,A,88aa45358e24f366cc6b64d81680b70c,21
4,A,817a4b6e475770881fcf98bb1adf39df,19
...,...,...,...
69497,A,bedc4a16130c08928c6277e535b19d28,1
69498,A,bedfe80d16a3c76b358285535e7a7b2d,1
69499,A,bee3b9ca480c8c14cebf1e05298aece2,1
69500,A,bee47d2b2747da68218d60adce3971ce,1


Из таблиц выше видим, что часть пользователей попала в 2 группы, так как мы знаем, что уникальных пользователей %n_users% у нас 68647.


Это может создать шум в данных, так как для AB тестов пользователи должны находится в одной определенной группе, не пересекаясь с другой.  
Поэтому я удалю юзеров %user_id%, которые попали в две группы одновременно. 

In [18]:
df_group_a = df[df['test_group'] == 'A']
df_group_a[['user_id', 'test_group']] 

,user_id,test_group
0,548e37dfa2690defe92201412ff9b02f,A
1,2c2205ffc0501665f64bf0703185b6d8,A
4,8a154919eefdcf48814a69da0b841e87,A
6,85e878f911af797a61126c49ef317509,A
7,269fec11aba1ba9189f637f26e20d98d,A
...,...,...
97212,011ed2ac9e21123f52e9f3088f163d23,A
97215,977807ddbb0300326446e863809e2ce9,A
97223,ded1303e92783ec4ee06ee1d47de4302,A
97228,3209ddc552fb40b183fade449f19ef85,A


In [19]:
df_group_b = df[df['test_group'] == 'B']
df_group_b[['user_id', 'test_group']]

,user_id,test_group
2,0b89952e0a145ffc36ea063e401e475c,B
3,6b6cc68c9a10b163c147b08be78f1cb2,B
5,8157d52e8757d4058408a793ea72841b,B
9,827212df88e9079d835d39955edf1ef5,B
11,ea01fd6087f469711185aa1fa8d67108,B
...,...,...
97222,6cf8151ea9ddafd3b9fae9eac5283dcf,B
97224,5745352852802c3ad3f6e67afbc9d6ce,B
97225,eed3ed6c01e59c37c6464e41bc066076,B
97226,33c9bb81b998ca0f1e8d24dd18273a54,B


Таблицы выше показывают пользовательские действия в каждой группе. Найдем пользователей, которые совершили действия в обоих группах

In [20]:
df_users_crossing = df_group_a.merge(df_group_b, left_on='user_id', right_on='user_id')
df_users_crossing

,datetime_of_event_x,user_id,event_x,success_flag_x,test_group_x,provider_x,amount_x,datetime_of_event_y,event_y,success_flag_y,test_group_y,provider_y,amount_y
0,2016-06-14 17:06:05.800,0aafa81ac7c2ad4676a90a8d62ab7b97,PaymentSuccess,1,A,QIWI Телеком,4.00,2016-06-11 18:07:12.800,PaymentSuccess,1,B,QIWI Онлайн,3509.00
1,2016-06-03 23:06:25.000,40422bb30b712d8990146596a877b7c7,PaymentSuccess,1,A,QIWI Мобайл,106.00,2016-06-12 22:21:11.600,PaymentSuccess,1,B,QIWI Мобайл,456.60
2,2016-06-03 23:06:25.000,40422bb30b712d8990146596a877b7c7,PaymentSuccess,1,A,QIWI Мобайл,106.00,2016-06-12 22:20:51.900,OpenPage,0,B,NaN,nan
3,2016-06-03 23:06:04.800,40422bb30b712d8990146596a877b7c7,OpenPage,0,A,NaN,nan,2016-06-12 22:21:11.600,PaymentSuccess,1,B,QIWI Мобайл,456.60
4,2016-06-03 23:06:04.800,40422bb30b712d8990146596a877b7c7,OpenPage,0,A,NaN,nan,2016-06-12 22:20:51.900,OpenPage,0,B,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,2016-06-14 20:27:32.300,febe34ccd8dc35d3d0577fcd45bff31c,OpenPage,0,A,NaN,nan,2016-06-15 06:33:46.100,OpenPage,0,B,NaN,nan
2908,2016-06-14 18:01:15.500,efb5af71c0517b78749f7fe6b2cbf89b,OpenPage,0,A,NaN,nan,2016-06-15 08:39:58.200,PaymentSuccess,1,B,QIWI Онлайн,1937.00
2909,2016-06-13 06:19:37.300,a761f92624ab2b6388753cd4e8499041,OpenPage,0,A,NaN,nan,2016-06-11 12:29:00.500,OpenPage,0,B,NaN,nan
2910,2016-06-13 06:19:37.300,a761f92624ab2b6388753cd4e8499041,OpenPage,0,A,NaN,nan,2016-06-10 07:40:37.500,OpenPage,0,B,NaN,nan


**2912 действий пользователей попали в группу A и B**

In [21]:
df = df[~df.user_id.isin(df_users_crossing.user_id)]
df

,datetime_of_event,user_id,event,success_flag,test_group,provider,amount
0,2016-06-13 23:31:57.800,548e37dfa2690defe92201412ff9b02f,OpenPage,0,A,NaN,nan
1,2016-06-07 04:14:20.700,2c2205ffc0501665f64bf0703185b6d8,OpenPage,0,A,NaN,nan
3,2016-06-16 12:35:21.800,6b6cc68c9a10b163c147b08be78f1cb2,OpenPage,0,B,NaN,nan
4,2016-06-14 09:42:25.000,8a154919eefdcf48814a69da0b841e87,OpenPage,0,A,NaN,nan
5,2016-06-10 10:40:06.800,8157d52e8757d4058408a793ea72841b,PaymentSuccess,0,B,NaN,nan
...,...,...,...,...,...,...,...
97224,2016-06-07 19:00:45.100,5745352852802c3ad3f6e67afbc9d6ce,PaymentSuccess,1,B,QIWI Онлайн,617.00
97225,2016-06-12 17:12:47.700,eed3ed6c01e59c37c6464e41bc066076,OpenPage,0,B,NaN,nan
97226,2016-06-15 05:40:57.200,33c9bb81b998ca0f1e8d24dd18273a54,OpenPage,0,B,NaN,nan
97227,2016-06-08 11:15:57.400,857fa51168a167166d244e760b6c0141,OpenPage,0,B,NaN,nan


Проверим уникальных пользователей

In [22]:
df.groupby(['test_group', 'user_id']).size().reset_index(name='count') \
      .sort_values(['count'], ascending=[False])                                         \
      .reset_index(drop = True)

,test_group,user_id,count
0,A,4118a432b767d136b1adb4d2a82ce32c,35
1,A,fa77b42f47b4b5ac8ec0803c04cd8298,22
2,A,88aa45358e24f366cc6b64d81680b70c,21
3,A,cd1a50c1af1b4089855e22dac3fd23f7,21
4,A,817a4b6e475770881fcf98bb1adf39df,19
...,...,...,...
67787,A,be2d5da66debfde02ad1e392d9358b4c,1
67788,A,be302badca2fbd08aa1a2783a1958374,1
67789,A,be36b6c96988fbf604311f0ebcaecec8,1
67790,A,be39158a0f001a9240b73f55109f6a3b,1


In [23]:
df['user_id'].unique().shape[0]

67792

Теперь датасет подготовлен к анализу

### Исследование данных 

Мы можем заметить, что **%event == PaymentSuccess% не всегда получает статус %success_flag ==1%**, возможно, существует проблема с транзакциями.  

Проблема может быть связана с каким-то определенным провайдером, но этих данных у нас нет, потому что данные о провайдере появляются только при статусе success_flag == 1), поэтому данная мысль остается предположением. 

In [24]:
df[(df['success_flag'] == 0) & (df['event'] == 'PaymentSuccess')][:5]

,datetime_of_event,user_id,event,success_flag,test_group,provider,amount
5,2016-06-10 10:40:06.800,8157d52e8757d4058408a793ea72841b,PaymentSuccess,0,B,NaN,nan
7,2016-06-14 08:59:56.300,269fec11aba1ba9189f637f26e20d98d,PaymentSuccess,0,A,NaN,nan
49,2016-06-06 19:23:37.900,88b3ff417c896ca5700dc2ffec743364,PaymentSuccess,0,B,NaN,nan
61,2016-06-10 11:47:11.600,110425c2d8f934aba347b3c3e65ce0ef,PaymentSuccess,0,B,NaN,nan
68,2016-06-13 19:09:11.500,986a9339b00c2a30095c0f3ca060578b,PaymentSuccess,0,B,NaN,nan


In [25]:
df[(df['success_flag'] == 0) & (df['event'] == 'PaymentSuccess')]['user_id'].shape[0]

6660

Мы тестируем варианты оплаты, которые появляются у провайдеров мобильной связи на последнем этапе оплаты. Думаю, что нет большой разницы у какого именно провайдера была произведена оплата.  

In [26]:
df[df['event'] == 'PaymentSuccess'] \
    .groupby(['test_group', 'provider']).size().reset_index(name='users_count')

,test_group,provider,users_count
0,A,QIWI Мобайл,2253
1,A,QIWI Онлайн,4855
2,A,QIWI Привет,3212
3,A,QIWI Телеком,4574
4,B,QIWI Мобайл,2130
5,B,QIWI Онлайн,4711
6,B,QIWI Привет,2992
7,B,QIWI Телеком,4149


#### Количество действий в группе по событиям %event%

In [27]:
df_user_events = df.groupby(['event', 'test_group']).size().reset_index(name='count')
df_user_events

,event,test_group,count
0,OpenPage,A,30931
1,OpenPage,B,27750
2,PaymentSuccess,A,18233
3,PaymentSuccess,B,17303


#### Количество действий в группе со статусом %success_flag%

In [28]:
df_users_actions = df.groupby(['success_flag', 'test_group']).size().reset_index(name='count')
df_users_actions

,success_flag,test_group,count
0,0,A,34270
1,0,B,31071
2,1,A,14894
3,1,B,13982


#### Общее количество действий %event% от пользователей A и B

In [29]:
group_a_events = df[df['test_group'] == 'A']['user_id'].shape[0]
group_a_events

49164

In [30]:
group_b_events = df[df['test_group'] == 'B']['user_id'].shape[0]
group_b_events

45053

### AB - тест

**Нулевая гипотеза**. Конверсия останется прежней после изменения формы оплаты  (задача — опровергнуть гипотезу).  
**Альтернативная гипотеза**. Изменения формы оплаты приведут к повышению конверсии.

**P-value**. Цель нахождения p-value как раз и состоит в том, чтобы определить, отличаются ли наблюдаемые результаты от ожидаемых настолько, чтобы можно было не отвергать «нулевую гипотезу» — гипотезу о том, что между экспериментальными переменными и наблюдаемыми результатами нет никакой связи.

**Уровень значимости a** - уровень значимости показывает, насколько мы уверены в наших результатах. Низкое значение для значимости соответствует низкой вероятности того, что экспериментальные результаты вышли случайными и наоборот.  
a = 0.05 или 5%


**Итого**: если **p − value** меньше нашего фиксированного **уровня значимости**, на котором мы проверяем гипотезу, то нулевую гипотезу следует отвергнуть, если более – то отвергать нулевую гипотезу оснований нет. 

**Примечание 1:** Как я заметила выше, остается вопросом, что считать за успешное действие: event == PaymentSuccess либо success_flag == 1.  
Я просчитаю оба показателя для AB теста.

**Примечание 2:** Я просчитываю конверсию по действиям в каждой группе, так как в этой задаче нас интересует сколько транзакций было в каждой группе, а не количество новых привлеченных пользователей.

In [31]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#### Вариант по полю %success_flag%

In [32]:
conversion_rates = df.groupby('test_group')['success_flag']

std_p = lambda x: np.std(x, ddof=0)              
se_p = lambda x: stats.sem(x, ddof=0)        

conversion_rates = conversion_rates.agg([np.mean, std_p, se_p])
conversion_rates.columns = ['conversion_rate', 'std_deviation', 'std_error']

conversion_rates

,conversion_rate,std_deviation,std_error
test_group,,,
A,0.303,0.460,0.002
B,0.310,0.463,0.002


### Вывод

https://www.evanmiller.org/ab-testing/chi-squared.html  
Новый экран оплаты конвертирует лучше и нулевая гипотеза отвергается при уровне значимости a == 0.05 (p_value = 0.0138)

#### Вариант по полю %event%

In [33]:
df_user_events = df_user_events[df_user_events['event'] == 'PaymentSuccess'].reset_index(drop = True)

df_user_events['total_events'] = [group_a_events, group_b_events]

df_user_events['conversion_rate'] = (df_user_events['count'] / df_user_events['total_events'])
df_user_events[['event', 'test_group', 'conversion_rate']]

,event,test_group,conversion_rate
0,PaymentSuccess,A,0.371
1,PaymentSuccess,B,0.384


### Вывод

Новый экран оплаты конвертирует лучше и нулевая гипотеза отвергается при уровне значимости a == 0.05 (p_value < 0.001)